In [1]:
from pymongo import MongoClient, DESCENDING
from scipy.sparse import *
from scipy import stats
# from sklearn import svm
# from sets import Set
import numpy as np
import json, sys, os, time, re, datetime, itertools, pickle

def logTime():
    return str(datetime.datetime.now())

In [2]:
local = False
if local:
    client = MongoClient('fc8iasm01', 27017)
    plan_col = client.aca.plan
    drug_col = client.aca.drug
else:
    client = MongoClient('ec2-52-53-173-200.us-west-1.compute.amazonaws.com', 27017)
    plan_col = client.plans.plans
    drug_col = client.formularies.drugs
    prov_col = client.providers.providers
    faci_col = client.providers.facilities

all_plan = plan_col.distinct('plan_id')
all_drug = drug_col.distinct('rxnorm_id')

print '%s: using %s Mongo, total drug: %d, total plan: %d, total provider: %d' %(
    logTime(), 'local' if local else 'aws', len(all_drug), len(all_plan), prov_col.count())
# client.formularies.scollection_names()
# client.providers.collection_names()

2016-07-31 09:23:16.945441: using aws Mongo, total drug: 46206, total plan: 0, total provider: 8799098


In [4]:
all_plan = plan_col.distinct('plan_id')

In [8]:
ak_plan = [i for i in all_plan if 'AK' in i]
len(ak_plan)

34

In [9]:
for p in plan_col.find({'plan_id':{'$in':ak_plan}}):
    print p,'\n'

{u'marketing_name': u'Blue Cross Blue Shield Plus 1500, a Multi-State Plan', u'network': [{u'network_tier': u'HERITAGE-PLUS'}], u'formulary': [{u'drug_tier': u'GENERIC', u'mail_order': True, u'cost_sharing': [{u'copay_amount': 10.0, u'pharmacy_type': u'1-MONTH-IN-RETAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}, {u'copay_amount': 30.0, u'pharmacy_type': u'3-MONTH-IN-MAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}]}, {u'drug_tier': u'PREFERRED-BRAND', u'mail_order': True, u'cost_sharing': [{u'copay_amount': 40.0, u'pharmacy_type': u'1-MONTH-IN-RETAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}, {u'copay_amount': 120.0, u'pharmacy_type': u'3-MONTH-IN-MAIL', u'copay_opt': u'NO-CHARGE', u'coinsurance_rate': 0.0, u'coinsurance_opt': u'NO-CHARGE'}]}, {u'drug_tier': u'NONPREFERRED-BRAND', u'mail_order': True, u'cost_sharing': [{u'copay_amount': 0.0, u'ph

In [34]:
prov_col.count({'plans.plan_id':{'$eq':'73836AK0790002'}})

1181

In [35]:
drug_col.count({'plans.plan_id':{'$eq':'73836AK0790002'}})

8809

In [162]:
%reload_ext autoreload
%autoreload 2
from query_drug_feature import *
from query_plan_feature import *
from query_provider_feature import *

In [33]:
for p in ak_plan:
    for f in getProviderListForPlans(prov_col, [p]):
        print p, len(f['npi'])

38344AK0620003 3066
38344AK0620004 3066
38344AK0620006 3066
38344AK0620008 3066
38344AK0570001 3066
38344AK0570002 3066
38344AK0980003 3066
38344AK0980006 3066
38344AK0980012 3066
74819AK0010007 56895
74819AK0010008 56895
74819AK0010009 56895
74819AK0010010 56895
74819AK0010011 56895
74819AK0010012 56895
74819AK0020003 56895
74819AK0020004 56895
74819AK0020005 56895
74819AK0020006 56895
73836AK0750001 952
73836AK0750002 1099
73836AK0750003 952
73836AK0750004 1099
73836AK0840001 1099
73836AK0850001 1099
73836AK0850002 952
73836AK0860001 1099
73836AK0860002 952
73836AK0790001 952
73836AK0790002 1099
21989AK0030001 350
21989AK0050001 143
21989AK0050002 143
21989AK0070001 350


In [143]:
prov_state = getProviderAllStates(prov_col, ak_plan)
one_s=getProviderStateForOnePlan(prov_col, '74819AK0020003')

In [159]:
# [(p['count'],p['key']) for p in one_s]
# prov_col.find_one({'plans.plan_id':{'$eq':'74819AK0010007'}})
# len(prov_state)
drug_state = getDrugAggregateAllStates(drug_col, ak_plan)
# one_s

In [156]:
ds=[[f['cnt'],f['key']] for f in getDrugAggregateCountForOnePlan(drug_col,'38344AK0620003')]
# drug_col.count({'plans.plan_id':{'$in':['74819AK0010007']}})

In [164]:
getProviderStateForOnePlan(prov_col, '74819AK0010009')

[{u'count': 1393, u'key': u'Periodontics', u'location': 8839},
 {u'count': 401, u'key': u'Prosthodontics', u'location': 1423},
 {u'count': 3238, u'key': u'Orthodontics', u'location': 18196},
 {u'count': 11, u'key': u'Oral Pathology', u'location': 103},
 {u'count': 2882, u'key': u'Oral Maxillofacial Surgery', u'location': 17956},
 {u'count': 1836, u'key': u'Endodontics', u'location': 9335},
 {u'count': 2371, u'key': u'Pediatric Dentistry', u'location': 11087},
 {u'count': 49050, u'key': u'General Dentistry', u'location': 195595}]

In [47]:
all_prov=prov_col.find({'plans.plan_id':{'$in':ak_plan}}).distinct('npi')
one_prov=prov_col.find({'plans.plan_id':{'$eq':'73836AK0790001'}}).distinct('npi')

In [48]:
import numpy as np
test = np.in1d(all_prov, one_prov, assume_unique=True)

In [82]:
sp1=csr_matrix(([1]*len(one_prov),([0]*len(one_prov),np.where(np.in1d(all_prov, one_prov, assume_unique=True))[0])),shape=(1,len(all_prov)))


In [167]:
all_plan = plan_col.distinct('plan_id')
np.unique([i[5:7] for i in all_plan])

array([u'', u'AK', u'AL', u'AR', u'AZ', u'CO', u'DC', u'DE', u'FL', u'GA',
       u'HI', u'IA', u'ID', u'IL', u'IN', u'KS', u'KY', u'LA', u'MA',
       u'ME', u'MI', u'MO', u'MS', u'MT', u'NC', u'ND', u'NE', u'NH',
       u'NJ', u'NM', u'NV', u'OH', u'OK', u'OR', u'PA', u'SC', u'SD',
       u'TN', u'TX', u'UT', u'VA', u'WA', u'WI', u'WV', u'WY', u'XX'], 
      dtype='<U2')

In [15]:
prov_col.find_one({'facility_name':{'$exists':True}})

{u'_id': ObjectId('577606dcc421d272dcd7deaa'),
 u'accepting': u'accepting',
 u'addresses': [{u'address': u'2235 DUBOIS DR',
   u'address_2': u'',
   u'city': u'WARSAW',
   u'phone': u'2604232567',
   u'state': u'IN',
   u'zip': u'46580'}],
 u'facility_name': u'',
 u'facility_type': [],
 u'gender': u'Male',
 u'languages': [u'SPANISH'],
 u'last_updated_on': u'2016-06-18',
 u'name': {u'first': u'ALAN R', u'last': u'GILBERT MD'},
 u'npi': u'1215961677',
 u'plans': [{u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200001',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200004',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200007',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200010',
   u'plan_id_type': u'HIOS-PLAN-ID'},
  {u'network_tier': u'PHP-SELECT',
   u'plan_id': u'50816IN0200013',
   u'plan_id_type': u'HIOS-PLAN-ID'},
 

In [42]:
for p in getProviderStateForPlans(prov_col, ['50816IN0200025','50816IN0230008']):
    print p['_id'], len(p['plan_states']), p['plan_states'][180]

50816IN0230008 630 {u'count': 10, u'location': 10, u'key': u'CARDIOVASCULAR DISEASE|CHINESE|'}
50816IN0200025 630 {u'count': 5, u'location': 5, u'key': u'HEMATOLOGY/ONCOLOGY| TELUGU|'}


In [49]:
for p in getProviderListForPlans(prov_col, ['23307AZ0400014','97667AZ0110007']):
    print len(p['npi'])

3286
987


In [48]:
all_npi = prov_col.find({'plans.plan_id':{'$in':az_plan}}).distinct('npi')
len(all_npi)

106230